### Project 1 - Unit Testing

Out code so far:

In [1]:
import itertools
import numbers
from datetime import timedelta, datetime
from collections import namedtuple


class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0:
            raise ValueError('Timezone name cannot be empty.')

        self._name = str(name).strip()
        # technically we should check that offset is a
        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError('Hour offset must be an integer.')

        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Minutes offset must be an integer.')

        if offset_minutes < -59 or offset_minutes > 59:
            raise ValueError('Minutes offset must between -59 and 59 (inclusive).')

        # for time delta sign of minutes will be set to sign of hours
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)

        # offsets are technically bounded between -12:00 and 14:00
        # see: https://en.wikipedia.org/wiki/List_of_UTC_time_offsets
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00.')

        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset

    @property
    def name(self):
        return self._name

    def __eq__(self, other):
        return (isinstance(other, TimeZone) and
                self.name == other.name and
                self._offset_hours == other._offset_hours and
                self._offset_minutes == other._offset_minutes)
    def __repr__(self):
        return (f"TimeZone(name='{self.name}', "
                f"offset_hours={self._offset_hours}, "
                f"offset_minutes={self._offset_minutes})")

In [75]:
class Account:
    transaction_counter = itertools.count(100)
    _interest_rate = 0.5  # percentage

    _transaction_codes = {
        'deposit': 'D',
        'withdraw': 'W',
        'interest': 'I',
        'rejected': 'X'
    }

    Confirmation = namedtuple('Confirmation', 'account_number, transaction_code, transaction_id, time_utc, time')

    def __init__(self, account_number, first_name, last_name, timezone=None, initial_balance=0):
        # in practice we probably would want to add checks to make sure these values are valid / non-empty
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name

        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        self.timezone = timezone

        self._balance = Account.validate_real_number(initial_balance, min_value=0)

    @property
    def account_number(self):
        return self._account_number

    @property
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        self.validate_and_set_name('_first_name', value, 'First Name')

    @property
    def last_name(self):
        return self._last_name

    @last_name.setter
    def last_name(self, value):
        self.validate_and_set_name('_last_name', value, 'Last Name')

    # also going to create a full_name computed property, for ease of use
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'

    @property
    def timezone(self):
        return self._timezone

    @property
    def balance(self):
        return self._balance

    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time zone must be a valid TimeZone object.')
        self._timezone = value

    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate

    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Interest rate must be a real number')
        if value < 0:
            raise ValueError('Interest rate cannot be negative.')
        cls._interest_rate = value

    def validate_and_set_name(self, property_name, value, field_title):
        if value is None or len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} cannot be empty.')
        setattr(self, property_name, value)

    @staticmethod
    def validate_real_number(value, min_value=None):
        if not isinstance(value, numbers.Real):
            raise ValueError('Value must be a real number.')

        if min_value is not None and value < min_value:
            raise ValueError(f'Value must be at least {min_value}')

        # validation passed, return valid value
        return value

    def generate_confirmation_code(self, transaction_code):
        # main difficulty here is to generate the current time in UTC using this formatting:
        # YYYYMMDDHHMMSS
        dt_str = datetime.now().strftime('%Y%m%d%H%M%S')
        return f'{transaction_code}-{self.account_number}-{dt_str}-{next(Account.transaction_counter)}'

    @staticmethod
    def parse_confirmation_code(confirmation_code, preferred_time_zone=None):
        # dummy-A100-20190325224918-101
        parts = confirmation_code.split('-')
        if len(parts) != 4:
            # really simplistic validation here - would need something better
            raise ValueError('Invalid confirmation code')

        # unpack into separate variables
        transaction_code, account_number, raw_dt_utc, transaction_id = parts

        # need to convert raw_dt_utc into a proper datetime object
        try:
            dt_utc = datetime.strptime(raw_dt_utc, '%Y%m%d%H%M%S')
        except ValueError as ex:
            # again, probably need better error handling here
            raise ValueError('Invalid transaction datetime') from ex

        if preferred_time_zone is None:
            preferred_time_zone = TimeZone('UTC', 0, 0)

        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError('Invalid TimeZone specified.')

        dt_preferred = dt_utc + preferred_time_zone.offset
        dt_preferred_str = f"{dt_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({preferred_time_zone.name})"

        return Confirmation(account_number, transaction_code, transaction_id, dt_utc.isoformat(), dt_preferred_str)

    def deposit(self, value):
        value = Account.validate_real_number(value, min_value=0.01)

        # get transaction code
        transaction_code = Account._transaction_codes['deposit']

        # generate a confirmation code
        conf_code = self.generate_confirmation_code(transaction_code)

        # make deposit and return conf code
        self._balance += value
        return conf_code

    def withdraw(self, value):
        value = Account.validate_real_number(value, min_value=0.01)
        accepted = False
        if self.balance - value < 0:
            # insufficient funds - we'll reject this transaction
            transaction_code = Account._transaction_codes['rejected']
        else:
            transaction_code = Account._transaction_codes['withdraw']
            accepted = True

        conf_code = self.generate_confirmation_code(transaction_code)

        # Doing this here in case there's a problem generating a confirmation code
        # - do not want to modify the balance if we cannot generate a transaction code successfully
        if accepted:
            self._balance -= value

        return conf_code

    def pay_interest(self):
        interest = self.balance * Account.get_interest_rate() / 100
        conf_code = self.generate_confirmation_code(Account._transaction_codes['interest'])
        self._balance += interest
        return conf_code

Testing happy path

In [76]:
a = Account('A100', 'Eric', 'Idle', timezone=TimeZone('MST', -7, 0), initial_balance=100)
print(a.balance)
print(a.deposit(150.02))
print(a.balance)
print(a.withdraw(0.02))
print(a.balance)
Account.set_interest_rate(1.0)
print(a.get_interest_rate())
print(a.pay_interest())
print(a.balance)
print(a.withdraw(1000))

100
D-A100-20260122135456-100
250.02
W-A100-20260122135456-101
250.0
1.0
I-A100-20260122135456-102
252.5
X-A100-20260122135456-103


In [77]:
import unittest

In [78]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

In [79]:
class TestAccount(unittest.TestCase):
    def setUp(self):
        print('Running setup...')
        self.x = 100

    def tearDown(self):
        print('Running teardown...')

    def test_ok(self):
        self.x = 200
        self.assertEqual(200, self.x)

    def test_2(self):
        self.assertEqual(200, self.x)


In [80]:
run_tests(TestAccount)

test_2 (__main__.TestAccount.test_2) ... FAIL
test_ok (__main__.TestAccount.test_ok) ... ok

FAIL: test_2 (__main__.TestAccount.test_2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_844/3744517060.py", line 14, in test_2
    self.assertEqual(200, self.x)
AssertionError: 200 != 100

----------------------------------------------------------------------
Ran 2 tests in 0.017s

FAILED (failures=1)


Running setup...
Running teardown...
Running setup...
Running teardown...


In [81]:
class TestAccount(unittest.TestCase):

    def test_create_timezone(self):
        tz = TimeZone('ABC', -1, -30)
        self.assertEqual('ABC', tz.name)
        self.assertEqual(timedelta(hours=-1, minutes=-30), tz.offset)

    def test_timezone_equal(self):
        tz1 = TimeZone('ABC', -1, -30)
        tz2 = TimeZone('ABC', -1, -30)
        self.assertEqual(tz1, tz2)

    def test_timezone_not_equal(self):
        tz = TimeZone('ABC', -1, -30)

        test_timezones = (
            TimeZone('DEF', -1, -30),
            TimeZone('ABC', -1, 0),
            TimeZone('ABC', 1, -30)
        )

        for i, test_tz in enumerate(test_timezones):
            with self.subTest(test_number=f'Test #{i}'):
                self.assertNotEqual(tz, test_tz)


In [82]:
run_tests(TestAccount)

test_create_timezone (__main__.TestAccount.test_create_timezone) ... ok
test_timezone_equal (__main__.TestAccount.test_timezone_equal) ... ok
test_timezone_not_equal (__main__.TestAccount.test_timezone_not_equal) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.016s

OK


In [104]:
class TestAccount(unittest.TestCase):

    def setUp(self):
        self.account_number = 'A100'
        self.first_name = 'FIRST'
        self.last_name = 'LAST'
        self.tz = TimeZone('TZ', 1, 30)
        self.balance = 100.00
        self.withdrawal_amount = 200

    def create_account(self):
        return Account(self.account_number, self.first_name, self.last_name, self.tz, self.balance)


    def test_create_timezone(self):
        tz = TimeZone('ABC', -1, -30)
        self.assertEqual('ABC', tz.name)
        self.assertEqual(timedelta(hours=-1, minutes=-30), tz.offset)

    def test_timezone_equal(self):
        tz1 = TimeZone('ABC', -1, -30)
        tz2 = TimeZone('ABC', -1, -30)
        self.assertEqual(tz1, tz2)

    def test_timezone_not_equal(self):
        tz = TimeZone('ABC', -1, -30)

        test_timezones = (
            TimeZone('DEF', -1, -30),
            TimeZone('ABC', -1, 0),
            TimeZone('ABC', 1, -30)
        )

        for i, test_tz in enumerate(test_timezones):
            with self.subTest(test_number=f'Test #{i}'):
                self.assertNotEqual(tz, test_tz)

    def test_create_account(self):

        a = self.create_account()

        self.assertEqual(self.account_number, a.account_number)
        self.assertEqual(self.first_name, a.first_name)
        self.assertEqual(self.last_name, a.last_name)
        self.assertEqual(self.first_name + ' ' + self.last_name, a.full_name)
        self.assertEqual(self.tz, a.timezone)
        self.assertEqual(self.balance, a.balance)

    def test_create_account_blank_first_name(self):

        self.first_name = ''

        with self.assertRaises(ValueError):
            a = self.create_account()

    def test_create_negative_balance(self):
        self.balance = -100.00

        with self.assertRaises(ValueError):
            a = self.create_account()

    def test_account_withdraw_ok(self):
        account_number = 'A100'
        first_name = 'FIRST'
        last_name = 'LAST'
        balance = 100.00
        withdrawal_amount = 20

        a = Account(account_number, first_name, last_name, initial_balance=balance)

        conf_code = a.withdraw(withdrawal_amount)
        self.assertTrue(conf_code.startswith('W-'))
        self.assertEqual(balance - withdrawal_amount, a.balance)

    def test_account_withdraw_overddraw(self):
        account_number = 'A100'
        first_name = 'FIRST'
        last_name = 'LAST'
        balance = 100.00
        withdrawal_amount = 200

        a = Account(account_number, first_name, last_name, initial_balance=balance)
        conf_code = a.withdraw(withdrawal_amount)

        self.assertTrue(conf_code.startswith('X-'))
        self.assertEqual(balance, a.balance)


In [105]:
run_tests(TestAccount)

test_account_withdraw_ok (__main__.TestAccount.test_account_withdraw_ok) ... ok
test_account_withdraw_overddraw (__main__.TestAccount.test_account_withdraw_overddraw) ... ok
test_create_account (__main__.TestAccount.test_create_account) ... ok
test_create_account_blank_first_name (__main__.TestAccount.test_create_account_blank_first_name) ... ok
test_create_negative_balance (__main__.TestAccount.test_create_negative_balance) ... ok
test_create_timezone (__main__.TestAccount.test_create_timezone) ... ok
test_timezone_equal (__main__.TestAccount.test_timezone_equal) ... ok
test_timezone_not_equal (__main__.TestAccount.test_timezone_not_equal) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.025s

OK
